In [4]:
import os
import pandas as pd
import csv


In [5]:
# Get the directory with the notebooks, including its subdirectories
current_dir = os.getcwd()

# Get the directories with data and results
data_dir = os.path.join(current_dir, "data/debloating")
temporary_dir= os.path.join(current_dir, "debloating_temporary_data")
results_dir = os.path.join(current_dir, "debloating_results")

project_dir = os.path.dirname(current_dir)

inputs_dir = os.path.join(project_dir, "inputs")

print(project_dir, current_dir, results_dir, data_dir, inputs_dir)

/home/rlefeuvr/Documents/Workspace/Debloat/bloat-energy-consumption /home/rlefeuvr/Documents/Workspace/Debloat/bloat-energy-consumption/notebooks /home/rlefeuvr/Documents/Workspace/Debloat/bloat-energy-consumption/notebooks/debloating_results /home/rlefeuvr/Documents/Workspace/Debloat/bloat-energy-consumption/notebooks/data/debloating /home/rlefeuvr/Documents/Workspace/Debloat/bloat-energy-consumption/inputs


In [11]:
# After we need to put together the PSYS and Size values for each program in each of the 3 different implementations

# Example usage
csv_with_ec = os.path.join(results_dir, 'averagePSYS_all_repeats.csv')

# Read the first CSV file
df = pd.read_csv(csv_with_ec, delimiter=';')
df['PSYS']= df['PSYS'].div(1000000).round(3)

df

Program Utilities   PSYS
0          date   bloated  0.288
1          date    chisel  0.284
2          date       cov  0.284
3          date     debop  0.284
4          grep   bloated  0.317
5          grep    chisel  0.457
6          grep       cov  0.315
7          grep     debop  0.315
8          gzip   bloated  0.335
9          gzip    chisel  0.549
10         gzip       cov  0.332
11         gzip     debop  0.332
12        mkdir   bloated  0.289
13        mkdir    chisel  0.288
14        mkdir       cov  0.288
15        mkdir     debop  0.288
16  printokens2   bloated  0.282
17  printokens2    chisel  0.281
18  printokens2       cov  0.285
19  printokens2     debop  0.282
20          sed   bloated  0.286
21          sed    chisel  0.289
22          sed       cov  0.290
23          sed     debop  0.287

In [13]:
df_pivot = df.pivot(index='Program', columns='Utilities')
df_pivot

PSYS                     
Utilities   bloated chisel    cov  debop
Program                                 
date          0.288  0.284  0.284  0.284
grep          0.317  0.457  0.315  0.315
gzip          0.335  0.549  0.332  0.332
mkdir         0.289  0.288  0.288  0.288
printokens2   0.282  0.281  0.285  0.282
sed           0.286  0.289  0.290  0.287

In [14]:
# Read the first CSV file
df = pd.read_csv("debloating_results/averagePSYS_with_std_all_repeats_pivot.csv", delimiter=';')

# Extract column names with "std" in their headers
std_columns = [col for col in df.columns if 'std' in col]

# Divide each column by 1,000,000
df.loc[:, std_columns] = df.loc[:, std_columns] / 1000000
df.loc[:, std_columns] = df.loc[:, std_columns].round(3)

df

Program  PSYS_avg_bloated  PSYS_avg_chisel  PSYS_avg_cov  \
0         date         287914.25        284160.70     284352.85   
1         grep         317086.10        457307.85     315260.85   
2         gzip         334755.55        549107.40     331841.20   
3        mkdir         289446.30        287996.70     287615.30   
4  printokens2         282000.05        281304.40     285134.15   
5          sed         285567.55        288768.75     290404.50   

   PSYS_avg_debop  PSYS_std_bloated  PSYS_std_chisel  PSYS_std_cov  \
0       284331.55             0.014            0.010         0.009   
1       315404.45             0.020            0.021         0.018   
2       331715.95             0.013            0.022         0.016   
3       287520.60             0.016            0.014         0.012   
4       281771.20             0.009            0.012         0.021   
5       287371.10             0.010            0.015         0.022   

   PSYS_std_debop  
0           0.010  
1           0.018  
2           0.008  
3           0.011  
4           0.013  
5           0.014

In [15]:
df_pivot['Difference_chisel'] = ((df_pivot['PSYS']['chisel'] - df_pivot['PSYS']['bloated']) / df_pivot['PSYS']['bloated']) * 100
df_pivot['Difference_cov'] = ((df_pivot['PSYS']['cov'] - df_pivot['PSYS']['bloated']) / df_pivot['PSYS']['bloated']) * 100
df_pivot['Difference_debop'] = ((df_pivot['PSYS']['debop'] - df_pivot['PSYS']['bloated']) / df_pivot['PSYS']['bloated']) * 100

new_df = pd.DataFrame({
                       'Chisel': df_pivot['Difference_chisel'].round(2),
                        'Debop': df_pivot['Difference_debop'].round(2),
                       'Cov': df_pivot['Difference_cov'].round(2)})

new_df


Chisel  Debop   Cov
Program                         
date          -1.39  -1.39 -1.39
grep          44.16  -0.63 -0.63
gzip          63.88  -0.90 -0.90
mkdir         -0.35  -0.35 -0.35
printokens2   -0.35   0.00  1.06
sed            1.05   0.35  1.40